## 1.1 Access SMAP data with Python

We will use the popular python `requests` library to bulk download SMAP data.

### Bulk data downloads from NSIDC

* We are going to do this in python, but you can do this in other languages also.

* Click [here](https://nsidc.org/support/faq/what-options-are-available-bulk-downloading-data-https-earthdata-login-enabled) to get additional instructions on bulk data downloads from NSIDC.

* SMAP data are available through the following url:
    * https://n5eil02u.ecs.nsidc.org/opendap/SMAP/

* For these examples, we will use **SPL3SMP version 004**, but the same methdology to access and subset SMAP data will work for any of the products availabe there

import packages

In [1]:
import calendar
import os

import requests

Create a directory to store our data downloads.

In [2]:
this_dir = os.getcwd()
this_dir

'g:\\Araki\\SMSigxSMAP\\0_code'

In [3]:
DATA_DIR = '../1_data/L3_SM_P'

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

#### Now we will create a function to return the url and filename of the data we want to download.

For SMAP we will do this by year, month, and day. The rest of the filename is predictable.

In [15]:
def SMAP_L3_P_36km_Path(year, month, day):
    fpath_start = 'https://n5eil01u.ecs.nsidc.org/SMAP/'
    host = 'https://n5eil01u.ecs.nsidc.org/'
    version = '.008'
    url_path = '{host}SMAP/SPL3SMP{version}/{year}.{month:02}.{day:02}/'.format(host=host,
                                                                                 version=version,
                                                                                 year=year,
                                                                                 month=month,
                                                                                 day=day)
    filename = 'SMAP_L3_SM_P_{year}{month:02}{day:02}_R18290_001.h5'.format(year=year, 
                                                                            month=month, 
                                                                            day=day)

    smap_data_path = url_path + filename
    
    return smap_data_path, filename

Add variables for username and password (edit to match your earthdata login credentials).

*Note: do not share your username and password with anyone!*

In [8]:
import json
my_credential_path = "./auth.json"
with open(my_credential_path, 'r') as infile:
    my_credentials = json.load(infile)
    
uid = my_credentials['username'] # Enter Earthdata Login user name
pswd = my_credentials['password'] # Enter Earthdata Login password
email = my_credentials['email'] # Enter Earthdata login email 

In [9]:
# Add Earthdata username and password here.
username = uid
password = pswd

assert username and password, 'You must supply your Earthdata username and password!'

Download March 2017 data

In [17]:
# Download data for March 2017
year = 2017
month = 3


def request_smap(year, month):
    # Use a requests session to keep track of authentication credentials
    _, days_in_month = calendar.monthrange(year, month)  # 31 days in March 2017
    with requests.Session() as session:
        session.auth = (username, password)
        
        for day in range(1, days_in_month + 1):
            print('Downloading SMAP data for: '+str(year)+'-'+str(month).zfill(2)+'-'+str(day).zfill(2))

            full_path, file_name = SMAP_L3_P_36km_Path(year, month, day)
            filepath = os.path.join(DATA_DIR, file_name)
            print(full_path)

            response = session.get(full_path)

            # If the response code is 401, we still need to authorize with earthdata.
            if response.status_code == 401:
                response = session.get(response.url)
                assert response.ok, 'Problem downloading data! Reason: {}'.format(response.reason)

            with open(filepath, 'wb') as f:
                f.write(response.content)

            print(file_name + ' downloaded')
            print('*** SM data saved to: '+ filepath +' *** ')

In [18]:
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
months = [1,2,3,4,5,6,7,8,9,10,11,12]

for year in years:
    for month in months: 
        try:
            request_smap(year, month)
        except:
            continue

https://n5eil01u.ecs.nsidc.org/SMAP/SPL3SMP.008/2015.01.01/SMAP_L3_SM_P_20150101_R18290_001.h5


AssertionError: Problem downloading data! Reason: Not Found

### Now you have the tools to download in bulk!

**FYI these scripts can take a long time, especially if you are downloading larger files.**
* SMAP 36km data are ~ 30MB
* SMAP 9km data are ~ 300MB
* SMAP 3km data are ~ 2 GB

** This is why OPENDAP makes more sense in some cases **